In [ ]:
import requests
import random
import time
import os

In [ ]:
BASE_URL = "https://www.kavak.com/mx/seminuevos"
PATH_TO_SAVE = '../data/raw_html/'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'es-MX,es;q=0.9'
}
os.makedirs(PATH_TO_SAVE, exist_ok=True)

In [ ]:
def get_raw_data(url, page):
    try:
        time.sleep(random.uniform(0.5, 1.5))
        response = requests.get(f"{url}?page={page}", headers=HEADERS, timeout=20)
        
        if response.status_code == 200:
            print(f"Pagina {page} descargada correctamente. ")
            return response
        else:
            print(f"Warning: La pagina {page} devolvio status {response.status_code}")
            return None
        
    except Exception as e:
        print(f"Sucedio un error {e} tratando de descargar la pagina numero: {page}")    
        return None

In [ ]:
def save_html(html_content, path, page):
    filename = f"{path}pagina_{page}.html"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content.text)

In [ ]:
def main(pages_limit=3):
    print(f"Iniciando crawler secuencial para {pages_limit} páginas...")
    
    for page in range(1, pages_limit + 1):
        response = get_raw_data(url=BASE_URL, page=page)

        if response and response.status_code == 200:
            save_html(response, path=PATH_TO_SAVE, page=page)
        else: 
            print(f"Saltando guardado de pagina {page} por un error.")
        

In [ ]:
if __name__ == "__main__":
    main(pages_limit=3)